# DataFrames

DataFrames are the workhorse of pandas and are directly inspired by the R programming language. We can think of a DataFrame as a bunch of Series objects put together to share the same index. Let's use pandas to explore this topic!

## Data Prep

In [20]:
import pandas as pd
import scipy as sp
import numpy as np
from pandas_datareader import data as pdr
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.linalg import inv, pinv
%matplotlib inline

In [21]:
def prep_zillow(mycsv,value_name):
    df = pd.read_csv('{0}.csv'.format(mycsv), encoding = "ISO-8859-1")
    if mycsv == 'City_Zhvi_3bedroom':
        df = df.drop('RegionID', axis=1)
    df = df.melt(id_vars=['RegionName', 'State', 'Metro', 'CountyName', 'SizeRank'], 
                             var_name='Year_Month', 
                             value_name=value_name
                            )
    df = df.set_index(['RegionName', 'State','Year_Month', 'Metro', 'CountyName'])
    df.sort_index(inplace=True)
    return df



In [22]:
def get_sp_500():
    sp500 = pdr.get_data_yahoo('^GSPC')
    sp500['year'] = sp500.index.year
    sp500['month'] = sp500.index.month
    return sp500

In [23]:
def total_return(prices):
    """Retuns the return between the first and last value of the DataFrame.
    Parameters
    ----------
    prices : pandas.Series or pandas.DataFrame
    Returns
    -------
    total_return : float or pandas.Series
        Depending on the input passed returns a float or a pandas.Series.
    """
    return prices.iloc[-1] / prices.iloc[0] - 1

In [24]:
def merge_data(city):
    home_price = prep_zillow(mycsv='City_Zhvi_3bedroom',value_name='home_price')
    home_rent = prep_zillow(mycsv='City_MedianRentalPrice_3Bedroom',value_name='home_rent')
    homes = home_price.merge(home_rent, how = 'left', on=['RegionName', 'Year_Month', 'State', 'CountyName', 'SizeRank', 'Metro'])
    homes.sort_index(inplace=True)
    homes = homes.fillna(0)
    homes['adj_price'] = homes['home_price'] + homes['home_rent']
    city = homes.loc[city]
    city = city.reset_index()
    city['return_realestate'] = (city.adj_price - city.adj_price.shift(1))/city.adj_price.shift(1)
    city['Year_Month'].loc[0].split("-")
    f = lambda x: x["Year_Month"].split("-")
    year_month = list(city.apply(f, axis=1))
    year_month = pd.DataFrame(year_month)
    year_month.columns = ['year', 'month']
    city = city.join(year_month, how='left')
    city.year = city.year.astype('int64')
    city.month = city.month.astype('int64')


    sp500 = get_sp_500()
    sp500 = pd.DataFrame(sp500.groupby(['year', 'month'], )['Adj Close'].apply(total_return))
    sp500['return_sp500'] = sp500['Adj Close']
    sp500 = sp500.reset_index()
    full_clean = city.merge(sp500, how='inner', left_on =['year','month'], right_on=['year', 'month'])
    final_clean = full_clean[['year', 'month', 'Metro', 'return_realestate', 'return_sp500']]
    return final_clean



## Modern Portfolio Theory

In [38]:
def mpt(df, desired_returns):
    returns = df[['return_realestate', 'return_sp500']]
    R = np.array(returns)
    stockMean = np.mean(R, axis=0)
    n_stock = len(returns.columns)
    
    def objFunction(W, R, target_ret):
        stock_mean = np.mean(R, axis=0)
        port_mean = np.dot(W, stock_mean) # portfolio mean
        cov = np.cov(R.T) # var-cov matrix
        port_var = np.dot(np.dot(W, cov), W.T) # portfolio variance
        penalty = 2000*abs(port_mean-target_ret) # penalty for deviation
        return np.sqrt(port_var) + penalty #objective function
    
    out_mean, out_std, out_weight = [], [], []
    for r in np.linspace(np.min(stockMean), np.max(stockMean), num=100):
        W = np.ones([n_stock])/n_stock
        b_ = [(0,1), (0,1)]
        c_ = ({'type': 'eq', 'fun': lambda W: sum(W) - 1})
        for i in range(n_stock):
            result=sp.optimize.minimize(objFunction, W, (R, r), method='SLSQP', constraints=c_, bounds=b_)
            if not result.success: 
                BaseException(result.message)

            out_mean.append(round(r, 4))
            std_=round(np.std(np.sum(R*result.x, axis=1)), 6)
            out_std.append(std_)
            out_weight.append(result.x)
            
        mean = pd.DataFrame(out_mean) 
        std = pd.DataFrame(out_std)
        weight = pd.DataFrame(out_weight)
        result1 = pd.merge(mean, std, left_index=True, right_index=True)
        result2 = pd.merge(result1, weight, left_index=True, right_index=True)
        result2.columns = ['mean', 'std', 'weight_realestate', 'weight_stocks']
        result2 = result2.round(4)
        result2['annual_mean'] = (1 + result2['mean'])**12 - 1
    
  
    return result2.loc[(result2['annual_mean'] > desired_returns - .01) & (result2['annual_mean'] < desired_returns + .01)].describe().T
        

In [26]:
def get_city_list():
    city_list = pd.read_csv('City_Zhvi_3bedroom.csv', encoding = "ISO-8859-1")
    return city_list['RegionName'].head(40)

In [27]:
def main(city, desired_returns):
    return_data = merge_data(city=city)
    return mpt(df=return_data, desired_returns=desired_returns)


In [28]:
def ui():
    print(get_city_list())
    city = input("City of Real Estate Investment (Single-Family Residence)):  ")
    desired_returns = float(input("What is your desired returns? Typically between 0.5 and .10 annually:  "))
    return main(city=city, desired_returns=desired_returns)
    

In [29]:
# ui()

## Analysis of Different Cities

In [39]:
main(city='Los Angeles', desired_returns=.08)

,count,mean,std,min,25%,50%,75%,max
mean,102.0,0.006457,0.000463,0.005700,0.006025,0.00650,0.006875,0.007200
std,102.0,0.023337,0.005338,0.014400,0.018700,0.02330,0.027925,0.032400
weight_realestate,102.0,0.363624,0.149433,0.111100,0.234825,0.36360,0.492375,0.616200
weight_stocks,102.0,0.636376,0.149433,0.383800,0.507625,0.63640,0.765175,0.888900
annual_mean,102.0,0.080309,0.005969,0.070586,0.074745,0.08085,0.085692,0.089905


In [40]:
main(city='Memphis', desired_returns=.08)

,count,mean,std,min,25%,50%,75%,max
mean,94.0,0.006445,0.000467,0.005700,0.006025,0.006400,0.006875,0.007200
std,94.0,0.024070,0.004907,0.015900,0.019800,0.024000,0.028300,0.032400
weight_realestate,94.0,0.343417,0.137762,0.111100,0.224725,0.343400,0.462075,0.575800
weight_stocks,94.0,0.656583,0.137762,0.424200,0.537925,0.656600,0.775275,0.888900
annual_mean,94.0,0.080152,0.006013,0.070586,0.074745,0.079562,0.085692,0.089905
